Заполните данные о необходимых переменных:

In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [2]:
Today = '31/10/24'

query = f"""
select MAX("reportDate") AS max_date
from "RISKACCESS"."bankAccountsBalanceDaily"
where "reportDate" <= TO_DATE('{Today}', 'DD/MM/YY')
"""
data_export = export_from_RISKCUSTOM(query)
Today = str(data_export.iloc[0,0])[:10]

print_to_excel = True
excel_tofolder_on_Z = True

Today

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


'2024-10-31'

Открываю следующие файлы:

In [3]:
query = f"""select * from "RISKACCESS"."bankAccountsBalanceDaily" where "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD')"""
data_export = export_from_RISKCUSTOM(query)
data_export.head(2)

,##batch_id,##deleted,##inserted,##origin_name,reportDate,holding,buCode,buCountryGroup,buCountryCode,buName,bankName,bankId,bankCountryCode,bankCountryGroup,accountCurrency,accountNumber,accountStatus,balance,balanceUsd,statementDate
0,41633746@WHPROD,N,2024-11-01 16:40:16,WHFLEX,2024-10-31,EUROCHEM,USBFC,(4) US,US,EuroChem North America Corp.,Vast,103902717,US,(4) US,USD,1087053,active,56718.62,56718.62,2024-10-15
1,41633746@WHPROD,N,2024-11-01 16:40:16,WHFLEX,2024-10-31,EUROCHEM,7100,(1) CH,CH,SUEK AG (Swiss),An Binh,ABBKVN,VN,(9) other,USD,1221027172080,active,72.92,72.92,2023-08-28


In [4]:
data_ecl = pd.read_excel('Поправка_по_ECL_Frozen_Cash_(гибкий).xlsx', sheet_name='Общая инф', header=3).drop(index=[5,6,7,8],axis=0)
data_ecl['impairment'] = data_ecl['Kusd.1']/data_ecl['Kusd']
data_ecl['impairment'] = data_ecl['impairment'].abs().astype('float64').round(3)

Произвожу расчеты:

In [5]:
data_work = data_export
data_work['Segment'] = merge_SalesUnits(data_work, col='buCode', merge_col='ocpSegment')
data_work['Period'] = '<1M'
data_work['impairment'] = data_work.merge(data_ecl, how='left', left_on=['buCode', 'bankName'], right_on=['code EK', 'Bank account'])['impairment'].fillna(0)
data_work['Balance_USD'] = data_work['balanceUsd']
data_work['balanceUSD_im'] = data_work['Balance_USD'] * (1-data_work['impairment'])
# Аня, еще небольшая корректировка, сделай порядок полей для кэша вот таким, пожалуйста:
data_work = data_work[['##batch_id','##deleted','##inserted','##origin_name','holding','buCountryGroup','buCountryCode','buCode','buName','bankName','bankId','bankCountryCode','bankCountryGroup','accountCurrency','accountNumber','accountStatus','balance','reportDate','balanceUSD_im','statementDate','Segment','Period','impairment','Balance_USD']]

for i in ['EUROCHEM', 'SUEK']:
    data_work_h = data_work[data_work['holding'] == i].reset_index(drop=True)
    if i == 'SUEK':
        data_work_SUEK = data_work_h
    if i == 'EUROCHEM':
        data_work_Ech = data_work_h

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['NAMSALES' 'EX SUEK AG' 'EX SUEK AG' ... 'SUEK RU' 'SUEK RU' 'SUEK RU']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']


Записываю данные в output file:

In [6]:
Output_file_SUEK = "_".join([str(date.today()), 'SUEK_Cash.xlsx'])
Output_file_Ech = "_".join([str(date.today()), 'Ech_Cash.xlsx'])
Sheet_in_output_file = 'Cash'
if excel_tofolder_on_Z == True:
    Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
    Output_file_SUEK = Output_path + Output_file_SUEK
    Output_file_Ech = Output_path + Output_file_Ech


if print_to_excel == True:
    data_work_SUEK.to_excel(Output_file_SUEK, sheet_name = Sheet_in_output_file, index = False)
    data_work_Ech.to_excel(Output_file_Ech, sheet_name = Sheet_in_output_file, index = False)